In [160]:
import pandas as pd
import os
from collections import defaultdict
from tqdm import tqdm

In [161]:
paths = os.listdir('data/posts')

In [162]:
bloggers_filter = defaultdict(int)

In [163]:
def clean_bloggers(path):
    df = pd.read_csv('data/posts/'+path)
    df = df[df['like_count']>=100]
    for user in df['username']:
        bloggers_filter[user] += 1

In [164]:
train_data = []

for path in tqdm(paths):
    clean_bloggers(path)

100%|██████████| 108/108 [00:00<00:00, 176.65it/s]


In [177]:
len(pd.read_csv('data/bloggers.csv', index_col='Unnamed: 0'))

3409

In [165]:
new_bloggers = pd.read_csv('data/bloggers.csv', index_col='Unnamed: 0')
new_bloggers['frequency'] = new_bloggers.index.map(lambda x: bloggers_filter.get(x, 0))
new_bloggers = new_bloggers[new_bloggers['frequency']>1]

In [166]:
new_bloggers.to_csv('data/new_bloggers.csv')

In [167]:
check = set(new_bloggers.index)

In [176]:
len(check)

578

In [168]:
def reformat(path):
    df = pd.read_csv('data/posts/'+path)
    df = df[df['username'].map(lambda x: x in check)]
    df = df.sort_values('taken_at')[['username','taken_at']]
    return list(zip(*map(df.get, df)))

In [169]:
datasets = []
for path in tqdm(paths):
    data = reformat(path)
    if len(data) > 10:
        datasets.append(data)

100%|██████████| 108/108 [00:00<00:00, 210.91it/s]


In [170]:
import pickle

In [171]:
with open('data/datasets.pkl', 'wb') as f:
    pickle.dump(datasets, f)

In [172]:
import pickle

with open('data/datasets.pkl', 'rb') as f:
    datasets = pickle.load(f)

In [174]:
from data.datasets import datasets